In [5]:
# Libraries

from selenium import webdriver
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time
import uuid
import re

In [6]:
# define classes of information to be collected

class Reviews:
    def __init__(self, review_id, item_id, review_time, review_variant, star_rating, review_content):
        self.review_id = review_id
        self.item_id=item_id
        self.review_time = review_time
        self.review_variant = review_variant
        self.star_rating  = star_rating
        self.review_content = review_content

class Items:
    def __init__(self, item_id, item_name, item_ratings, ratings_num, item_sold, shop_name, item_price, item_url, item_desc, image_url):
        self.item_id=item_id
        self.item_name=item_name
        self.item_ratings=item_ratings
        self.ratings_num=ratings_num
        self.item_sold=item_sold
        self.shop_name=shop_name
        self.item_price=item_price
        self.item_url=item_url
        self.item_desc=item_desc
        self.image_url=image_url


In [7]:
#Create Chrome options
chrome_options= Options()
chrome_options.add_argument('--no sandbox')
chrome_options.add_argument('disable-notifications')
chrome_options.add_argument('disable-inforbars')

# Set the path to the Chrome driver executable
chrome_driver_path = 'D:\Chrome web driver\chromedriver-win64\chromedriver.exe'

# Create an instance of the Chrome browser
browser = webdriver.Chrome(executable_path=chrome_driver_path, options=chrome_options)

# Navigate to a URL 
browser.get("https://shopee.com.my/")

C:\Users\Asus\AppData\Local\Temp\ipykernel_10680\1499064468.py:11: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(executable_path=chrome_driver_path, options=chrome_options)


In [8]:
soup=BeautifulSoup(browser.page_source, "html.parser")

the_links=[]

parent_div = soup.findAll('div', class_="col-xs-2-4 shopee-search-item-result__item")

for parent_div in parent_div:
    a=parent_div.find('a')
    if (a!=None):
        href=a.get('href')
        the_links.append(href)

        # print the links
        print(href)
        

The history saving thread hit an unexpected error (OperationalError('database or disk is full')).History will not be written to the database.
/99800mAh-Car-Power-bank-High-power-With-Pump-Power-Bank-Car-Jump-Start-Emergency-Light-i.173134281.7005003808?sp_atk=27af42a9-d210-44fa-a374-350e52b18091&xptdk=27af42a9-d210-44fa-a374-350e52b18091
/TAYAR-TYRE-MAXXIS-MAXIS-DIAMOND-BUNGA-MA3D-MA-3D-TUBELESS-60-80-17-70-80-17-70-90-17-80-90-17-NEW-i.232871686.6618595637?sp_atk=c9913c72-e985-4627-b5df-f32f64f5d6ef&xptdk=c9913c72-e985-4627-b5df-f32f64f5d6ef
/SEAT-SAKORN-DRAG-THAILAND-LC-135-Y15-Y16-125ZR-LAGENDA-NMAX-NVX-EGO-LC-SOLARIZ-AVANTIZ-EX5-RS150-VARIO-BEAT-VF3i-RAIDER-i.230894337.6570044748?sp_atk=7e34317b-307a-49ab-ae8d-30931db9a09f&xptdk=7e34317b-307a-49ab-ae8d-30931db9a09f
/Perfume-Initial-Original-Peach-Nectar-i.80502900.5005396733?sp_atk=9a7ecbf5-3152-4603-9dc3-09d376c9d6e6&xptdk=9a7ecbf5-3152-4603-9dc3-09d376c9d6e6
/SANTINO-988VF-Water-Jet-Cordless-Car-Wash-Floor（10-Years-Warranty）Spray

In [9]:
len(the_links)

30

In [10]:
items_list=[]
reviews_list=[]

In [12]:
for i in range (0,len(the_links)):
    
    # open link
    links='https://shopee.com.my'+ the_links[i]
    browser.get(links)
    time.sleep(10)

    # parse html using beautifulsoup
    soup=BeautifulSoup(browser.page_source, "html.parser")

    # extract product information
    id=uuid.uuid4()
    title = soup.find('div', attrs={'class':'_44qnta'}).find('span').get_text(strip=True)
    ratings=soup.find('div', class_="_1k47d8 _046PXf").get_text(strip=True)
    ratings_num=soup.select('div._1k47d8:not(div._1k47d8._046PXf)')[0].get_text(strip=True)
    sold=soup.find('div', class_="e9sAa2").get_text(strip=True)
    shop_name=soup.find('div',class_="VlDReK").get_text(strip=True)
    price = soup.find('div', class_='pqTWkA').get_text(strip=True)

    # extract description
    descriptions = soup.find_all('p', class_='irIKAp')
    

    desc_array=[]
    for description in descriptions:
        desc = re.sub(r'<[^>]+>', '', str(description))  # Remove HTML tags
        desc = desc.strip()  # Remove leading/trailing whitespaces
        desc_array.append(desc)

    text = "\n".join(desc_array)

    # extract picture url
    img_tag= soup.find('img', class_='_7D4JtJ')

    if img_tag:
        image = img_tag['src']
    else:
        image=None

    # append item to a list
    items_list.append(Items(id,title,ratings,ratings_num,sold,shop_name, price, links, text, image))
    print(text)
    
    # reviews extraction
    elements = browser.find_elements(By.CLASS_NAME,"product-rating-overview__filter")
    counter=5
    for j in range(1,6):
        # select star categories
        elements[j].click()
        time.sleep(2)

        # loop is for each category
        for k in range (0,31): 
            soup=BeautifulSoup(browser.page_source, "html.parser")
            review_containers=soup.findAll('div',class_="shopee-product-rating__main")

            no_reviews_found = False

            for rc in review_containers:
                review_id=uuid.uuid4()
                item_id=id
                time_var=rc.find('div',class_="shopee-product-rating__time").get_text(strip=True).split("|")

                if(len(time_var)==2):
                    review_time=time_var[0]
                    variant=time_var[1]
                else:
                    review_time=time_var[0]
                    variant="None"

                star_rating=counter
                reviews=rc.find('div',class_="Rk6V+3")

                if(reviews!=None):
                    reviews=reviews.get_text(strip=True)
                else:
                    reviews="None"
                    no_reviews_found = True

                reviews_list.append(Reviews(review_id, item_id, review_time,variant,star_rating,reviews))
            
            # no more reviews break
            if no_reviews_found:
                print("No reviews found, breaking the loop")
                break

            time.sleep(1)
            try:
                WebDriverWait(browser, 1).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[class^='shopee-icon-button shopee-icon-button--right "))).click()
            except:
                print("not found")
                break
            time.sleep(1)
        
        counter=counter-1
        time.sleep(2)
    print(i) 
    
    

- Very convenient .can be charge at home and car.
- Short circuit, over-charge, and over-heat protection,very safe.
- Starting a vehicle approximately up to 30 times After fully charged.
- This jump starter can start Gasoline car below 6.0L
- Start car at one time on cold &amp; hot areas.( on -20°C~60°C areas)
- 4 USB Charging ports support for charging various electronic devices.
- Highlight LED lights+flashing signal light+SOS rescue lights + Red and blue warning lights, convenient for field inspections, camping and other night activities.

Car Jumper Specification:

- Battery capacity:99800mAh
- Size:160*80*30mm
- Weight:340g
- USB Output:5V/1A
- Jump Start:12V
- Input:15V/1A / 2A
- Full charging time:about 3 hours
- Starting current:300A
- Peak current:600A
- Operating temperature: -40~80℃


KeyboardInterrupt: 

In [73]:
items_df = pd.DataFrame([vars(obj) for obj in items_list])
reviews_df = pd.DataFrame([vars(obj) for obj in reviews_list])

In [74]:
items_df.shape

(60, 10)

In [75]:
items_df.head()

,item_id,item_name,item_ratings,ratings_num,item_sold,shop_name,item_price,item_url,item_desc,image_url
0,926555e9-6ae8-4af5-bf6e-0a83ae1c503d,READY STOCK💝WEBEE Hipster Men Women's Kasut Sa...,4.9,3.2k,8.7k,WEBEE,RM5.00 - RM8.46,https://shopee.com.my/READY-STOCK💝WEBEE-Hipste...,[📱Follow WEBEE ❤ ( ◠‿◠ )❤\n🎒READY STOCK IN MA...,https://down-my.img.susercontent.com/file/711c...
1,882f686e-e22c-4c27-9afd-227770c58a37,L&F Women thick Heels sandals perempuan kasut ...,4.8,4.5k,10.5k,L&F Enterprise Malaysia,RM17.59,https://shopee.com.my/L-F-Women-thick-Heels-sa...,[💟WELCOME TO L&F SHOP! THANK YOU!\n\n🔥🔥Stock r...,https://down-my.img.susercontent.com/file/34c5...
2,08974ab6-0ef3-47cd-832d-d0420faac333,Crocss Jibbitz Cartoon Cute Animals 2.5D DIY S...,4.9,287,3.1k,bulongjier.my,RM0.91,https://shopee.com.my/Crocss-Jibbitz-Cartoon-C...,"[Pay attention to the store, receive coupons a...",https://down-my.img.susercontent.com/file/6775...
3,a5ef5c1b-0d64-4874-a2ee-3f2ddf91f3de,Women's Sneakers Shoes Sport Shoes Flat Kasut ...,4.8,1.9k,4.5k,easystore_2u,RM11.90,https://shopee.com.my/Women's-Sneakers-Shoes-S...,[#Sandal #Shoe #womenshoe #readystock #slippe...,https://down-my.img.susercontent.com/file/6916...
4,cae59307-4b40-4241-8119-9922c77c0a83,READY STOCKWEBEE HD-6008 Pinco Women's Flat Sh...,4.9,8.9k,21.8k,WEBEE,RM5.00 - RM9.50,https://shopee.com.my/READY-STOCKWEBEE-HD-6008...,[Follow WEBEE ( )\nREADY STOCK IN MALAYSIA + F...,https://down-my.img.susercontent.com/file/a266...


In [76]:
reviews_df.shape

(14334, 6)

In [77]:
reviews_df.tail()

,review_id,item_id,review_time,review_variant,star_rating,review_content
14329,820bd9c6-b80c-478d-b2d8-2109a0508f66,8c4a89e4-2cfa-4b5b-8186-37cb4de3272c,2020-04-23 11:18,"Variation: WHITE,38",1,Tidak memuaskan ....Pesan 3 tapi yg datang cum...
14330,ca9f7c07-64c3-41d4-92f5-b52ff402cad5,8c4a89e4-2cfa-4b5b-8186-37cb4de3272c,2022-07-26 10:38,"Variation: WHITE,39",1,Ok lah...
14331,a0eb4740-ba8c-41ae-a320-ff210a8d2596,8c4a89e4-2cfa-4b5b-8186-37cb4de3272c,2023-08-14 13:47,"Variation: BLACK,41",1,None
14332,9caf1c14-ac6c-47b7-89e9-7ebce10aa1e3,8c4a89e4-2cfa-4b5b-8186-37cb4de3272c,2023-06-22 06:55,"Variation: PINK,39",1,None
14333,b3fdbc19-cd25-42f0-ae53-c41178829a6f,8c4a89e4-2cfa-4b5b-8186-37cb4de3272c,2023-06-18 23:27,"Variation: WHITE,40",1,None


In [78]:
reviews_df.to_json('womenshoes_reviews_5.json', orient='records', lines=True, force_ascii=False, default_handler=str)
items_df.to_json('womenshoes_items_5.json', orient='records', lines=True, force_ascii=False, default_handler=str)

# Check Data

In [20]:
def combine(json_files):
    dataframes=[]
    for file in json_files:
        # Load JSON file into a DataFrame
        df = pd.read_json(file, lines=True)
        # Append DataFrame to the list
        dataframes.append(df)
    combined_df = pd.concat(dataframes, ignore_index=True)

    return combined_df


In [42]:
json_files = ["womenshoes_items_1.json", "womenshoes_items_2.json", "womenshoes_items_3.json","womenshoes_items_4.json","womenshoes_items_5.json"]
json_reviews=["womenshoes_reviews_1.json","womenshoes_reviews_2.json","womenshoes_reviews_3.json","womenshoes_reviews_4.json","womenshoes_reviews_5.json"]

# Save the combined data to a new JSON file
all_items_df=combine(json_files)
all_reviews_df=combine(json_reviews)

all_items_df.to_json('all_womenshoes_item.json', orient='records', lines=True, force_ascii=False, default_handler=str)
all_reviews_df.to_json('all_womenshoes_review.json',orient='records', lines=True, force_ascii=False, default_handler=str)